# Testing and experimenting with Siddon alg

In [12]:
from math import floor, ceil, sqrt

def raytrace2(x1, y1, z1, x2, y2, z2, dx, dy, dz, pix_numx, pix_numy, pix_numz):

    #first, compute starting and ending parametric values in each dimension
    if (x2 - x1) != 0:
        ax_0 = -x1/(x2 - x1)
        ax_N = (pix_numx*dx - x1)/(x2 - x1)
    else:
        ax_0 = 0
        ax_N = 1
    if (y2 - y1) != 0:
        ay_0 = -y1/(y2 - y1)
        ay_N = (pix_numy*dy - y1)/(y2 - y1)
    else:
        ay_0 = 0
        ay_N = 1
    if (z2 - z1) != 0:
        az_0 = -z1/(z2 - z1)
        az_N = (pix_numz - z1)/(z2 - z1)
    else:
        az_0 = 0
        az_N = 1
        
    #then calculate absolute max and min parametric values
    a_min = max(0, min(ax_0, ax_N), min(ay_0, ay_N), min(az_0, az_N))
    a_max = min(1, max(ax_0, ax_N), max(ay_0, ay_N), max(az_0, az_N))

    #now find range of indices corresponding to max/min a values
    if (x2 - x1) >= 0:
        i_min = ceil(pix_numx - (pix_numx*dx - a_min*(x2 - x1) - x1)/dx)
        i_max = floor((x1 + a_max*(x2 - x1))/dx)
    elif (x2 - x1) < 0:
        i_min = ceil(pix_numx - (pix_numx*dx - a_max*(x2 - x1) - x1)/dx)
        i_max = floor((x1 + a_min*(x2 - x1))/dx)

    if (y2 - y1) >= 0:
        j_min = ceil(pix_numy - (pix_numy*dy - a_min*(y2 - y1) - y1)/dy)
        j_max = floor((y1 + a_max*(y2 - y1))/dy)
    elif (y2 - y1) < 0:
        j_min = ceil(pix_numy - (pix_numy*dy - a_max*(y2 - y1) - y1)/dy)
        j_max = floor((y1 + a_min*(y2 - y1))/dy)
    
    if (z2 - z1) >= 0:
        k_min = ceil(pix_numz - (pix_numz*dz - a_min*(z2 - z1) - z1)/dz)
        k_max = floor((z1 + a_max*(z2 - z1))/dz)
    elif (z2 - z1) < 0:
        k_min = ceil(pix_numz - (pix_numz*dz - a_max*(z2 - z1) - z1)/dz)
        k_max = floor((z1 + a_min*(z2 - z1))/dz)

    #next calculate the list of parametric values for each coordinate
    a_x = []
    if (x2 - x1) > 0:
        for i in range(i_min, i_max):
            a_x.append((i*dx - x1)/(x2 - x1))
    elif (x2 - x1) < 0:
        for i in range(i_min, i_max):
            a_x.insert(0, (i*dx - x1)/(x2 - x1))

    a_y = []
    if (y2 - y1) > 0:
        for j in range(j_min, j_max):
            a_y.append((j*dy - y1)/(y2 - y1))
    elif (y2 - y1) < 0:
        for j in range(j_min, j_max):
            a_y.insert(0, (j*dy - y1)/(y2 - y1))

    a_z = []
    if (z2 - z1) > 0:
        for k in range(k_min, k_max):
            a_z.append((k*dz - z1)/(z2 - z1))
    elif (z2 - z1) < 0:
        for k in range(k_min, k_max):
            a_z.insert(0, (k*dz - z1)/(z2 - z1))

    #finally, form the list of parametric values
    a_list = [a_min] + a_x + a_y + a_z + [a_max]
    a_list = list(set(a_list))
    a_list.sort()
    return a_list

def calc_midpoints(x1, y1, z1, x2, y2, z2, a_list):
    '''Calculates the midpoints of the ray sections that intersect each voxel'''
    # loop though, computing midpoints for each adjacent pair of a values for chosen coord
    i_mid = []
    for m in range(1, len(a_list)):
        x = .5 * (a_list[m] + a_list[m - 1]) * (x2 - x1) + x1
        y = .5 * (a_list[m] + a_list[m - 1]) * (y2 - y1) + y1
        z = .5 * (a_list[m] + a_list[m - 1]) * (z2 - z1) + z1
        i_mid.append((x, y, z))
    return i_mid

def intersect_length(x1, y1, z1, x2, y2, z2, a_list):
    # find length of intersection by multiplying difference in parametric values by total ray length
    lengths = []
    for m in range(1, len(a_list)):
        lengths.append(sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2) * (a_list[m] - a_list[m - 1]))
    return lengths

In [2]:
# Assemble arguments
x1, y1, z1 = 0.2, 0, 0
x2, y2, z2 =0.7, 1, 1
dx, dy, dz = 0.1, 0.1, 0.1
nx, ny, nz = 10, 10, 10
args = (x1, y1, z1, x2, y2, z2, dx, dy, dz, 0, nx, 0, ny, 0, nz)
args2 = (x1, y1, z1, x2, y2, z2, dx, dy, dz, nx, ny, nz)

# Test Siddon
# a_list = raytrace(*args)
a_list = raytrace2(*args2)
midpoints = calc_midpoints(*(list(args[:6])+[a_list]))
lengths = intersect_length(*(list(args[:6])+[a_list]))
print('Midpoints: ' + str([(round(x[0], 3), round(x[1], 3), round(x[2], 3)) for x in midpoints]))
print('Lengths: ' + str([round(x, 3) for x in lengths]))
print(len(midpoints))

Midpoints: [(0.225, 0.05, 0.05), (0.275, 0.15, 0.15), (0.3, 0.2, 0.2), (0.325, 0.25, 0.25), (0.375, 0.35, 0.35), (0.4, 0.4, 0.4), (0.425, 0.45, 0.45), (0.475, 0.55, 0.55), (0.525, 0.65, 0.65), (0.575, 0.75, 0.75), (0.625, 0.85, 0.85), (0.675, 0.95, 0.95)]
Lengths: [0.15, 0.15, 0.0, 0.15, 0.15, 0.0, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15]
12


In [14]:
from math import floor, ceil, sqrt
def siddon_params(start, stop, vox_pitch, vox_count):
    x1, y1, z1 = start
    x2, y2, z2 = stop
    dx, dy, dz = vox_pitch
    pix_numx, pix_numy, pix_numz = vox_count

    #first, compute starting and ending parametric values in each dimension
    if (x2 - x1) != 0:
        ax_0 = -x1/(x2 - x1)
        ax_N = (pix_numx*dx - x1)/(x2 - x1)
    else:
        ax_0 = 0
        ax_N = 1
    if (y2 - y1) != 0:
        ay_0 = -y1/(y2 - y1)
        ay_N = (pix_numy*dy - y1)/(y2 - y1)
    else:
        ay_0 = 0
        ay_N = 1
    if (z2 - z1) != 0:
        az_0 = -z1/(z2 - z1)
        az_N = (pix_numz - z1)/(z2 - z1)
    else:
        az_0 = 0
        az_N = 1
        
    #then calculate absolute max and min parametric values
    a_min = max(0, min(ax_0, ax_N), min(ay_0, ay_N), min(az_0, az_N))
    a_max = min(1, max(ax_0, ax_N), max(ay_0, ay_N), max(az_0, az_N))

    #now find range of indices corresponding to max/min a values
    if (x2 - x1) >= 0:
        i_min = ceil(pix_numx - (pix_numx*dx - a_min*(x2 - x1) - x1)/dx)
        i_max = floor((x1 + a_max*(x2 - x1))/dx)
    elif (x2 - x1) < 0:
        i_min = ceil(pix_numx - (pix_numx*dx - a_max*(x2 - x1) - x1)/dx)
        i_max = floor((x1 + a_min*(x2 - x1))/dx)

    if (y2 - y1) >= 0:
        j_min = ceil(pix_numy - (pix_numy*dy - a_min*(y2 - y1) - y1)/dy)
        j_max = floor((y1 + a_max*(y2 - y1))/dy)
    elif (y2 - y1) < 0:
        j_min = ceil(pix_numy - (pix_numy*dy - a_max*(y2 - y1) - y1)/dy)
        j_max = floor((y1 + a_min*(y2 - y1))/dy)
    
    if (z2 - z1) >= 0:
        k_min = ceil(pix_numz - (pix_numz*dz - a_min*(z2 - z1) - z1)/dz)
        k_max = floor((z1 + a_max*(z2 - z1))/dz)
    elif (z2 - z1) < 0:
        k_min = ceil(pix_numz - (pix_numz*dz - a_max*(z2 - z1) - z1)/dz)
        k_max = floor((z1 + a_min*(z2 - z1))/dz)

    #next calculate the list of parametric values for each coordinate
    a_x = []
    if (x2 - x1) > 0:
        for i in range(i_min, i_max):
            a_x.append((i*dx - x1)/(x2 - x1))
    elif (x2 - x1) < 0:
        for i in range(i_min, i_max):
            a_x.insert(0, (i*dx - x1)/(x2 - x1))

    a_y = []
    if (y2 - y1) > 0:
        for j in range(j_min, j_max):
            a_y.append((j*dy - y1)/(y2 - y1))
    elif (y2 - y1) < 0:
        for j in range(j_min, j_max):
            a_y.insert(0, (j*dy - y1)/(y2 - y1))

    a_z = []
    if (z2 - z1) > 0:
        for k in range(k_min, k_max):
            a_z.append((k*dz - z1)/(z2 - z1))
    elif (z2 - z1) < 0:
        for k in range(k_min, k_max):
            a_z.insert(0, (k*dz - z1)/(z2 - z1))

    #finally, form the list of parametric values
    a_list = [a_min] + a_x + a_y + a_z + [a_max]
    a_list = list(set(a_list))
    a_list.sort()
    return a_list

def siddon_midpoints(start, stop, a_list):
    '''Calculates the midpoints of the ray sections that intersect each voxel'''
    ray_mid = 0.5 * (stop - start)
    # loop though, computing midpoints for each adjacent pair of a values for chosen coord
    i_mid = []
    for m in range(1, len(a_list)):
        (x, y, z) = (a_list[m] + a_list[m - 1]) * ray_mid
        i_mid.append((x, y, z))
    return i_mid

def new_siddon_midpoints(start, stop, a_list):
    '''Calculates the midpoints of the ray sections that intersect each voxel'''
    ray_mid = 0.5 * (stop - start)
    # loop though, computing midpoints for each adjacent pair of a values for chosen coord
    i_mid = []
    for m in range(1, len(a_list)):
        # find halfway between params
        i_mid_param = (0.5 * (a_list[m] - a_list[m - 1]) + a_list[m - 1])
        (x, y, z) = i_mid_param * (stop - start) + start
        # print(f"new: with param {i_mid_param}, coord is {(x, y, z)}")
        (xold, yold, zold) = (a_list[m] + a_list[m - 1]) * ray_mid
        # print(f"old: {(xold, yold, zold)}")
        # print(f"a_list[m] + a_list[m - 1] = {a_list[m] + a_list[m - 1]} for m = {m}")
        # print(f"Difference is {np.array((x, y, z)) - np.array((xold, yold, zold))}")
        i_mid.append((x, y, z))
    return i_mid

def vol_indices(midpoints, vox_pitch):
    '''Identifies the voxels for which the midpoints belong
        - shifts down by 0.5, then rounds to nearest integer'''
    dx, dy, dz = vox_pitch
    i_mid = []
    for (x,y,z) in midpoints:
        x_ix = round((x-0.5*dx)/dx)
        y_ix = round((y-0.5*dy)/dy)
        z_ix = round((z-0.5*dz)/dz)
        i_mid.append((x_ix,y_ix,z_ix))
    return i_mid

def siddon_lengths(start, stop, a_list):
    # find length of intersection by multiplying difference in parametric values by total ray length
    entire_length = np.linalg.norm(stop - start)
    lengths = []
    for m in range(1, len(a_list)):
        lengths.append(entire_length * (a_list[m] - a_list[m - 1]))
    return lengths


In [5]:
import numpy as np
x1, x2, x3 = np.array([1.3, 2, 3])

In [8]:
dx, dy, dz = 0.1, 0.1, 0.1
nx, ny, nz = 10, 10, 10
start = np.array([0.2,0,0])
stop = np.array([0.7,1,1])
my_list = siddon_params(start, stop, [0.1]*3 , [10]*3)

In [ ]:
my_lengths = siddon_lengths(start, stop, my_list)

In [ ]:
my_midpoints = siddon_midpoints(start, stop, my_list)
my_midpoints_new = new_siddon_midpoints(start, stop, my_list)

In [23]:
x1, y1, z1 = 0.2, 0.5, 1
x2, y2, z2 =0.7, 1, 0
dx, dy, dz = 0.1, 0.1, 0.1
nx, ny, nz = 10, 10, 10
args2 = (x1, y1, z1, x2, y2, z2, dx, dy, dz, nx, ny, nz)

a_list = raytrace2(*args2)
midpoints = calc_midpoints(x1, y1, z1, x2, y2, z2, a_list)

In [24]:
start = np.array([0.2,0.5,1])
stop = np.array([0.7,1,0])
my_list = siddon_params(start, stop, [0.1]*3 , [10]*3)
my_midpoints_new = new_siddon_midpoints(start, stop, my_list)

In [25]:
np.array(my_midpoints_new[:]) - np.array(midpoints[:])

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

# Comparing midpoint methods

In [41]:
def siddon_mid0(start, stop, a_list):
    '''OG method
    Calculates the midpoints of the ray sections that intersect each voxel'''
    x1, y1, z1 = start
    x2, y2, z2 = stop
    # loop though, computing midpoints for each adjacent pair of a values for chosen coord
    i_mid = []
    for m in range(1, len(a_list)):
        x = .5 * (a_list[m] + a_list[m - 1]) * (x2 - x1) + x1
        y = .5 * (a_list[m] + a_list[m - 1]) * (y2 - y1) + y1
        z = .5 * (a_list[m] + a_list[m - 1]) * (z2 - z1) + z1
        i_mid.append((x, y, z))
    return i_mid

def siddon_mid1(start, stop, a_list):
    '''
    Calculates the midpoints of the ray sections that intersect each voxel'''
    ray_mid = 0.5 * (stop - start)
    # loop though, computing midpoints for each adjacent pair of a values for chosen coord
    i_mid = []
    for m in range(1, len(a_list)):
        (x, y, z) = (a_list[m] + a_list[m - 1]) * ray_mid
        i_mid.append((x, y, z))
    return i_mid

def siddon_mid2(start, stop, a_list):
    '''Logically correct method
    Calculates the midpoints of the ray sections that intersect each voxel'''
    # loop though, computing midpoints for each adjacent pair of a values for chosen coord
    i_mid = []
    for m in range(1, len(a_list)):
        # find halfway between params
        i_mid_param = (0.5 * (a_list[m] - a_list[m - 1]) + a_list[m - 1])
        (x, y, z) = i_mid_param * (stop - start) + start
        i_mid.append((x, y, z))
    return i_mid

In [27]:
def compare(l1, l2):
    return np.array(l1) - np.array(l2)

In [42]:
start = np.array([0.2,0.5,1])
stop = np.array([0.7,1,0])
# start = np.array([0.2,0,0])
# stop = np.array([0.7,1,1])
my_list = siddon_params(start, stop, [0.1]*3 , [10]*3)
l0 = siddon_mid0(start, stop, my_list)
l1 = siddon_mid1(start, stop, my_list)
l2 = siddon_mid2(start, stop, my_list)

siddon_mid0 and siddon_mid2 seem to do the same thing, unlike siddon_mid1

In [44]:
# compare(l0, l1)
compare(l0, l2)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])